# Assignment 1

Veera Anand, Aarav Dewangan, Samuel Martinez Koss, and John Melel.

Notes:

- Questions 01, 02, 03, 04 from ``0001-01-02-02_performance-metrics_homework.html``

- Bonus Question from ``0001-01-01-02_bias_vs_variance_homework.html``

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

base_dir_sam = '/content/drive/MyDrive/[96] MS-ADS/ADSP 31018 -- Machine Learning II/adsp-31018-machine-learning-2/data'

base_dir = base_dir_sam
os.chdir(base_dir)
!pwd

/content/drive/.shortcut-targets-by-id/1IrvmCPGR8kslT0s_vcGNpM2PQGQBbV_L/adsp-31018-machine-learning-2/data


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.4f}'.format

## Question 01

Datasets ``binomial-classification-v1.csv`` and ``binomial-classification-v2.csv`` contain actuals and predictions values from a binomial response variable.

1. Print one record containing a true/positive, true/negative, false/positive, false/negative

2. Manually compute accuracy, precision, recall, f1_Score

3. Compute the metrics in item 2 using sklearn and tensorflow, and torch

4. Show the values matching across all three computing approaches

In [ ]:
binom1 = pd.read_csv('binomial-classification-v1.csv')
binom2 = pd.read_csv('binomial-classification-v2.csv')

In [ ]:
binom01 = pd.concat([binom1, binom2], ignore_index=True)

### 1.

In [ ]:
neg = binom01.loc[binom01['predicted'] == 0]
pos = binom01.loc[binom01['predicted'] == 1]

TP = pos.loc[pos['actual'] == 1]
TN = neg.loc[neg['actual'] == 0]
FP = pos.loc[pos['actual'] == 0]
FN = neg.loc[neg['actual'] == 1]

print("Example Record Containing a True/Positive:\n", TP.sample(), "\n")
print("Example Record Containing a True/Negative:\n", TN.sample(), "\n")
print("Example Record Containing a False/Positive:\n", FP.sample(), "\n")
print("Example Record Containing a False/Negative:\n", FN.sample())

Example Record Containing a True/Positive:
        Unnamed: 0  actual  predicted
15472        5473       1          1 

Example Record Containing a True/Negative:
        Unnamed: 0  actual  predicted
13076        3077       0          0 

Example Record Containing a False/Positive:
       Unnamed: 0  actual  predicted
8335        8335       0          1 

Example Record Containing a False/Negative:
       Unnamed: 0  actual  predicted
7077        7077       1          0


### 2.

### 3.

### 4.

## Question 02

Dataset ``binomial-classification-v3.csv`` contain actuals and probability prediction values from a binomial response variable.

1. Manually compute the log loss

2. Compute the log loss using sklearn, tensorflow, and torch

3. Show the values matching across all three computing approaches

In [ ]:
binom3 = pd.read_csv('binomial-classification-v3.csv')

## Question 03

Dataset ``regression-v1.csv`` contains actuals and predictions from a normally distributed response variable with a varying mean but constant variance.

1. Manually compute mean absolute error, mean squared error, r2, explained variance score, and mean absolute percentage error

2. Compute the metrics in item 2 using sklearn and tensorflow, and torch

3. Document that the values match across all three computing approaches

In [ ]:
reg1 = pd.read_csv('regression-v1.csv')

NameError: name 'pd' is not defined

## Question 04

Dataset ``multinomial-classification-v1.csv`` contains actuals and predictions for a multinomial response variable. Manually compute the performance and check that it matches in sklearn, tensorflow, and torch.

Helper functions and other code modified from ``0001-01-02-01_performance-metrics_lecture.html``

In [4]:
multinom1 = pd.read_csv('multinomial-classification-v1.csv')

In [16]:
from sklearn.metrics import accuracy_score, log_loss
import torch
import torch.nn.functional as F
import tensorflow as tf

In [17]:
actuals = multinom1[['y1', 'y2', 'y3']].to_numpy()
predictions = multinom1[['p1', 'p2', 'p3']].to_numpy()
num_classes = predictions.shape[1]

actuals_pt = torch.tensor(actuals).clone().detach()
predictions_pt = torch.tensor(predictions).clone().detach()

actuals_tf = tf.convert_to_tensor(actuals, dtype=tf.float32)
predictions_tf = tf.convert_to_tensor(predictions, dtype=tf.float32)

### Accuracy By Category

In [7]:
# Corrected Manual Accuracy per Class Calculation
def accuracy_per_class(y_true, y_pred):
    """
    Computes accuracy per class (class-wise accuracy) based on thresholded predictions.
    """
    return np.array([np.mean(y_true[:, i] == np.round(y_pred[:, i])) for i in range(y_true.shape[1])])

In [24]:
acc_per_class = accuracy_per_class(actuals, predictions)
print(f"Manually computed Accuracies by Category: {acc_per_class.tolist()}")

accuracies_sklearn = [accuracy_score(actuals[:, i], predictions[:, i].round()) for i in range(num_classes)]
print(f"Scikit-learn Accuracies by Category     : {accuracies_sklearn}")

accuracies_pt = []
for i in range(num_classes):
    # predicted_i >= 0.5 => predicted as "1" for class i
    pred_binary = (predictions_pt[:, i] >= 0.5).float()
    # actuals[:, i] is 1 or 0 if truly one-hot
    acc_i = (pred_binary == actuals_pt[:, i]).float().mean()
    accuracies_pt.append(acc_i.item())
rounded_accuracies_pt = [round(acc, 4) for acc in accuracies_pt]
print(f"PyTorch Accuracies by Category          : {rounded_accuracies_pt}")

accuracies_tf = []
for i in range(num_classes):
    pred_binary = tf.cast(predictions_tf[:, i] >= 0.5, tf.float32)
    acc = tf.reduce_mean(tf.cast(pred_binary == actuals_tf[:, i], tf.float32))
    accuracies_tf.append(acc.numpy())
converted_accuracies = [round(float(acc), 4) for acc in accuracies_tf]
print(f"TensorFlow Accuracies by Category       : {converted_accuracies}")

Manually computed Accuracies by Category: [0.867, 0.871, 0.857]
Scikit-learn Accuracies by Category     : [0.867, 0.871, 0.857]
PyTorch Accuracies by Category          : [0.867, 0.871, 0.857]
TensorFlow Accuracies by Category       : [0.867, 0.871, 0.857]


### Overall Accuracy

In [14]:
# Corrected Manual Overall Accuracy
def overall_accuracy(y_true, y_pred):
    """
    Computes overall accuracy.
    """
    correct = np.sum(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))
    return correct / len(y_true)

In [20]:
overall_acc = overall_accuracy(actuals, predictions)
print(f"Manually computed Overall Accuracy: {overall_acc:.4f}")

overall_accuracy_sklearn = accuracy_score(np.argmax(actuals, axis=1),
                                          np.argmax(predictions, axis=1))
print(f"Scikit-learn Overall Accuracy     : {overall_accuracy_sklearn:.4f}")

pred_labels = predictions_pt.argmax(dim=1)
true_labels = actuals_pt.argmax(dim=1)
overall_accuracy_pt = (pred_labels == true_labels).float().mean().item()
print(f"PyTorch Overall Accuracy          : {overall_accuracy_pt:.4f}")

y_true_labels = tf.argmax(actuals_tf, axis=1)
y_pred_labels = tf.argmax(predictions_tf, axis=1)
accuracy_metric = tf.keras.metrics.Accuracy()
accuracy_metric.update_state(y_true_labels, y_pred_labels)
overall_accuracy_tf = accuracy_metric.result().numpy()

print(f"TensorFlow Overall Accuracy       : {overall_accuracy_tf:.4f}")

Manually computed Overall Accuracy: 0.7940
Scikit-learn Overall Accuracy     : 0.7940
PyTorch Overall Accuracy          : 0.7940
TensorFlow Overall Accuracy       : 0.7940


### Log Loss

In [12]:
# Corrected Manual Log Loss
def log_loss(y_true, y_pred):
    """
    Computes log loss for multi-class classification.
    """
    epsilon = 1e-15  # Small value to avoid log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred)) / len(y_true)

In [21]:
log_loss_value = log_loss(actuals, predictions)
print(f"Manually computed Log Loss: {log_loss_value:.4f}")

log_loss_sklearn = log_loss(actuals, predictions)
print(f"Scikit-learn Log Loss     : {log_loss_sklearn:.4f}")

labels_int = actuals_pt.argmax(dim=1)  # shape (N,)
predictions_clamped = predictions_pt.clamp(min=1e-7, max=1.0 - 1e-7)  # avoid log(0)
log_loss_pt = F.nll_loss(predictions_clamped.log(), labels_int)
print(f"PyTorch Log Loss          : {log_loss_pt.item():.4f}")

log_loss_metric = tf.keras.losses.CategoricalCrossentropy()
log_loss_tf = log_loss_metric(actuals, predictions).numpy()
print(f"TensorFlow Log Loss       : {log_loss_tf:.4f}")

Manually computed Log Loss: 0.5372
Scikit-learn Log Loss     : 0.5372
PyTorch Log Loss          : 0.5372
TensorFlow Log Loss       : 0.5372


## Bonus Question

Replicate Figure 7.1 from the textbook Elements of Statistical Learning in Python. Reference the available R code from the repository: https://github.com/alanjeffares/elements-of-statistical-learning. While it is acceptable to use Gen AI tools to generate the Python code, you must review the generated code for correctness and disclose the Gen AI tool used. Additionally, discuss any issues encountered and highlight errors that were present in the original or generated code. The final code should successfully reproduce the chart to closely resemble the original Figure 7.1.